In [18]:
import nltk

In [19]:
def swn(textlist):
    """
    given list of words, return a list of the sentiment value of each word
    """
    from nltk.corpus import sentiwordnet

    # first tag all the words with appropriate pos
    poslist = nltk.pos_tag(textlist)
    swnvalues = list()
    for w in poslist:
        postagged = 0
        if 'NN' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'n'))
        elif 'JJ' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'a'))
        elif 'RB' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'r'))
        elif 'VB' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'v'))
        else:
            postagged = 1

        # then calculate the value of each word
        try:
            if postagged == 0:
                value = word[0].pos_score() - word[0].neg_score()
            else:
                value = 0

        except:
            continue
        # store the value for this word
        swnvalues.append((w[0],value))

    return swnvalues


def swnsum(swnlist):
    """
    given list of (word,swnvalues), return the normalized swn value of the text

    a positive value means more good things are said than bad things
    a negative value means more bad than good
    each company's reviews can be compared with other review set values
    """
    swnsum = 0.0
    for i in swnlist:
        swnsum = swnsum + i[1]

    return (swnsum / len(swnlist))

In [20]:
sentimentScores = []
import pandas as pd
data=pd.read_csv("C:\\Users\\dream_000\\Desktop\\SYS 6018 Machine Learning (R)\\final project\\telecom_set2.csv",encoding="ISO-8859-1")
text=data.text
text=pd.DataFrame(text)

text.to_csv('telecom_set2_text_only.csv')

In [21]:
sentimentScores = []
with open('C:\\Users\\dream_000\\Desktop\\SYS 6018 Machine Learning (R)\\final project\\telecom_set2_text_only.csv') as f:
    for line in f:
        tokens = line.split()
        stopwords = nltk.corpus.stopwords.words('english')
        parsed = [w.lower() for w in tokens if w.isalpha()]
        parsed = [w for w in parsed if w not in stopwords]
        poslist = nltk.pos_tag(parsed)
        for i in range(len(poslist)):
            if 'VB' in poslist[i][1]:
                poslist[i] = (nltk.stem.WordNetLemmatizer().lemmatize(poslist[i][0], 'v'),'VB')
            parsed[i] = poslist[i][0]
        #textlist = analtext.parser()
        swnlist = swn(parsed)
        #value = swnsum(swnlist)
        swnsum = 0.0
        for i in swnlist:
            swnsum = swnsum + i[1]
        #sentimentScores.append(swnsum / len(swnlist))
        if len(swnlist) == 0:
            continue
        else:
            value = swnsum /len(swnlist)
            sentimentScores.append(value)
        #print('Senti-wordnet value: %.7f' % (value * 100))


In [22]:
s = pd.Series(sentimentScores)

result = pd.concat([data, s], axis=1)
result
result.to_csv('telecom_set2_result.csv')